In [100]:
import string
import numpy as np

In [101]:
def one_hot(n_classes: int, idx: int) -> np.ndarray:
    vector = np.zeros(n_classes)
    vector[idx] = 1.0
    return vector

def str2seq(word: str, alphabet: list) -> np.ndarray:
    return np.array([one_hot(len(alphabet), alphabet.index(c)) for c in word])

def seq2str(seq: np.ndarray, alphabet: list) -> str:
    return ''.join([alphabet[np.argmax(x)] for x in seq])

In [102]:
alphabet = ' .,:;\"\'' + string.ascii_letters
n_symbols = len(alphabet)
print(n_symbols, alphabet)

59  .,:;"'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ


In [103]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y / np.sum(y)

In [104]:
def calculate_attention(x: np.ndarray, M: np.ndarray) -> np.ndarray:
    scores = softmax(M @ x)
    values = scores.reshape((scores.size, 1)) * M
    return np.sum(values, axis=0)

In [105]:
class RNN:
    def __init__(self, n_in: int, n_l: int, n_h: int, n_out: int) -> None:
        self.W_xh: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.W_hh: np.ndarray = np.random.uniform(-1, 1, (n_l, n_h, n_h))

        self.W_ho: np.ndarray = np.random.uniform(-1, 1, (n_out, n_h))
        self.W_oh: np.ndarray = np.random.uniform(-1, 1, (n_h, n_out))

    def encode(self, sequence: np.ndarray) -> tuple[np.ndarray]:
        context = np.zeros(len(self.W_xh))
        for x in sequence:
            attention = calculate_attention(context, self.W_hh @ context)
            context = np.tanh(self.W_xh @ x + attention)
        return context
    
    def decode(self, context: np.ndarray, t: int) -> tuple[np.ndarray]:
        outputs = np.zeros((t, len(self.W_ho)))
        out = np.zeros(len(self.W_ho))

        for i in range(t):
            attention = calculate_attention(context, self.W_hh @ context)
            context = np.tanh(self.W_oh @ out + attention)
            out = softmax(self.W_ho @ context)
            outputs[i] = out.copy()

        return (outputs, context)

In [138]:
in_text = 'the quick brown fox jumps over the lazy dog'
sequence = str2seq(in_text, alphabet)

model = RNN(n_symbols, 4, 16, n_symbols)
context = model.encode(sequence)
outputs, context = model.decode(context, 128)

out_text = seq2str(outputs, alphabet)
print(out_text)

KdY;evMnYSsQLLLLLLSLL;RsW'ZCuEwiY;n uHE Uooyc:glfqSeZfaZlIECE'iwmshqSLLLSssQbmRsL;qsc;aFkMXdY;K;;;ShLICEw w;qXNhuEooS WZcXhqYvMR
